In [1]:
#!pip install packagename
# importing modules
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import os
from os import chdir as cd
import time
import fiona

C:\Users\uttar\AppData\Local\Temp\ipykernel_2212\776568913.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
US_places = gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Maps_2020\compiled.shp')
US_counties = gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Maps_2020\tl_2020_us_county.zip')
US_states = gpd.read_file(r'D:\Work\Box Sync\Trends_all states\Maps_2020\tl_2020_us_state.zip')

# source: https://gis.stackexchange.com/questions/141580/which-projection-is-best-for-mapping-the-contiguous-united-states
US_states = US_states.to_crs('EPSG:9311')
US_places = US_places.to_crs('EPSG:9311')
US_counties = US_counties.to_crs('EPSG:9311')

US_places.shape, US_counties.shape, US_states.shape

((32188, 17), (3234, 18), (56, 15))

In [3]:
US_places.groupby(['STATEFP'])['GEOID'].count().sum()

32188

In [4]:
from pathlib import Path
# define the file location   CT_scernario_2020_22_tractssp4_total_2050
scenario = ['ssp1', 'ssp2', 'ssp3', 'ssp4', 'ssp5']
folder = Path(r"D:\Work\Box Sync\Trends_all states\Spatial_Population_geotiff\Population from SSP\r_extract_output\\")
for i, s in enumerate(scenario):
    print(s)
    # reading the zip file
    csv_filenames = scenario[i] + '_total_*.tif.csv'
    filenames = folder.glob(csv_filenames)
    
    # combining places for US into one file
    master_df = pd.DataFrame()
    for filename in filenames:
        # print(filename)
        df = pd.read_csv(filename, encoding='latin-1').add_suffix(str(filename)[-13:-8])
        master_df = pd.concat([master_df, df], axis =1)
        # master_df.append(pd.concat([pd.read_csv(filename, index_col = 0, dtype = {'GEOID': str})], axis =1).pipe(pd.DataFrame).add_suffix(str(filename)[-8:-4]))

    print(master_df['population_2020'].sum())
    # master_df.shape
    # master_df.isna().sum()
    # master_df[master_df['GEOID_2010'] != master_df['GEOID_2100']]

    master_df['GEOID'] = master_df['GEOID_2100'].astype(str).str.rjust(7,'0')
    master_df.drop(master_df.filter(regex='GEOID_|NAME_|NAMELSAD_20').columns, axis =1, inplace = True)

    # master_df[master_df['std_2020'] > 2500]
    master_df.to_csv(r'D:\Work\Box Sync\Trends_all states\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\\population_from_' + str(s) + '.csv')


ssp1
239515351.66600987
ssp2
238873196.472252
ssp3
234495987.376714
ssp4
237511094.644772
ssp5
243379092.7204964


In [5]:
# merging all scenarios

In [6]:
from pathlib import Path
# define the file location   CT_scernario_2020_22_tractssp4_total_2050
folder = Path(r"D:\Work\Box Sync\Trends_all states\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\\")
ssp_filenames = 'population_from_*' + '.csv'
files = folder.glob(ssp_filenames)
dfs = [pd.read_csv(f, index_col = 0, dtype = {'GEOID': str},) for f in files]

# Source: https://stackoverflow.com/questions/48236438/merge-multiple-dataframe-with-specified-suffix
suffix = ('_ssp1', '_ssp2', '_ssp3', '_ssp4', '_ssp5')
for i in range(len(dfs)):
    dfs[i] = dfs[i].add_suffix(suffix[i])

from functools import reduce
def agg_df(dfList):
    temp=reduce(lambda left, right: pd.merge(left, right, 
                                             left_index=True, right_index=True, 
                                             how='outer'), dfList)
    return temp

df = agg_df(dfs)
df.shape
df.to_csv(r"D:\Work\Box Sync\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\NCAR_agg_scenarios.csv")

In [7]:
df.filter(regex= 'population').sum().to_csv(r"D:\Work\Box Sync\Trends_all states\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\NCAR_agg_scenarios_totals.csv")

In [25]:
# df.filter(regex= 'population').sum()

In [13]:
df_IL = df[df['GEOID_ssp5'].str.startswith('17')].reset_index(drop=True)
df_IL

,population_2010_ssp1,meanpop_2010_ssp1,std_2010_ssp1,population_2020_ssp1,meanpop_2020_ssp1,std_2020_ssp1,population_2030_ssp1,meanpop_2030_ssp1,std_2030_ssp1,population_2040_ssp1,...,meanpop_2080_ssp5,std_2080_ssp5,population_2090_ssp5,meanpop_2090_ssp5,std_2090_ssp5,NAMELSAD_2100_ssp5,population_2100_ssp5,meanpop_2100_ssp5,std_2100_ssp5,GEOID_ssp5
0,944.900452,315.499969,217.382980,970.977173,324.206940,223.382172,991.429626,331.035950,228.087433,998.587952,...,518.492126,357.246826,1618.806763,540.515686,372.421265,Sheldon village,1658.177490,553.661499,381.478912,1769212
1,748.365234,109.072502,108.482796,765.320923,111.543755,110.940697,777.652405,113.341042,112.728264,780.735291,...,173.156815,172.220642,1232.374756,179.615768,178.644669,Karnak village,1257.808472,183.322678,182.331543,1739077
2,250.621918,89.944885,62.359680,265.061066,95.126900,65.952423,279.362671,100.259560,69.510948,291.807800,...,155.608353,107.884819,456.345062,163.776199,113.547661,Mound City city,473.465729,169.920578,117.807625,1750751
3,264.019287,65.894760,52.432930,266.375244,66.482765,52.982212,267.227448,66.695465,53.235165,264.672607,...,94.251663,76.035919,387.027557,96.595551,78.046875,New Grand Chain village,391.055573,97.600876,78.975067,1752467
4,641.960388,48.530254,66.515205,639.070129,48.311760,66.215736,632.647339,47.826218,65.550255,616.590271,...,62.174084,85.215332,833.360291,62.999508,86.346657,Olmsted village,833.039185,62.975231,86.313385,1755899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,317.341034,92.837418,18.957458,347.963074,101.795830,20.786772,379.599762,111.051071,22.676702,409.613037,...,199.050125,40.646160,732.291138,214.230164,43.745937,Whiteash CDP,774.398987,226.548721,46.261391,1781165
1462,159.057861,118.304001,71.479958,173.128723,128.769623,77.803345,187.643265,139.565247,84.326126,201.207077,...,247.686478,149.653595,357.539490,265.930603,160.676804,White Heath CDP,377.188019,280.544769,169.506760,1781295
1463,1448.529907,243.020615,100.976303,1597.520752,268.016876,111.362373,1752.175415,293.963379,122.143280,1900.805420,...,531.500549,220.841171,3416.615723,573.207397,238.170578,Willowbrook CDP,3620.296631,607.379028,252.369080,1781938
1464,1123.514404,99.610535,93.735100,1233.291382,109.343338,102.893723,1346.887817,119.414772,112.376274,1455.029907,...,214.682083,202.017914,2607.352051,231.167252,217.531479,Willow Lake CDP,2758.585205,244.575546,230.150696,1782029


In [27]:
(df_IL.filter(regex= 'population').sum()/1000000).to_csv(r"D:\Work\Box Sync\Trends_all states\Spatial_Population_geotiff\Population from SSP\r_concat_SSP\NCAR_agg_scenarios_totals_IL.csv")